# XProj demo

In [1]:
import numpy as np
import shapely
import xarray as xr
import xvec

import xproj

xr.set_options(display_expand_indexes=True);

## Example datasets

- Vector data cube (https://xvec.readthedocs.io) with no CRS attached

In [2]:
rng = np.random.default_rng(1234)
random_points = shapely.points(rng.uniform(-180, 180, 10), rng.uniform(-90, 90, 10))

ds_vec = xr.Dataset(coords={"points": ("space", random_points)}).set_xindex(
    "points", xvec.GeometryIndex
)

ds_vec

<xarray.Dataset> Size: 80B
Dimensions:  (space: 10)
Coordinates:
  * points   (space) object 80B POINT (171.61191601133118 -10.618898030367944...
Dimensions without coordinates: space
Data variables:
    *empty*
Indexes:
    points   GeometryIndex (crs=None)

## Set the CRS

`.proj.assign_crs()` can be used to set the CRS: it creates a new scalar coordinate (if it doesn't exist yet) with a `xproj.CRSIndex`.

Note: the name of the spatial reference coordinate is abritrary ("spatial_ref" is a common name).

In [3]:
ds_wgs84 = ds_vec.proj.assign_crs(spatial_ref="epsg:4326")

ds_wgs84

<xarray.Dataset> Size: 88B
Dimensions:      (space: 10)
Coordinates:
  * points       (space) object 80B POINT (171.61191601133118 -10.61889803036...
  * spatial_ref  int64 8B 0
Dimensions without coordinates: space
Data variables:
    *empty*
Indexes:
    points       GeometryIndex (crs=None)
    spatial_ref  CRSIndex (crs=EPSG:4326)

## Get the CRS

Via the `.proj.crs` property, which returns a [pyproj.CRS](https://pyproj4.github.io/pyproj/stable/api/crs/crs.html) object.

In [4]:
ds_wgs84.proj.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

Alternatively, it is possible to explicitly pass the CRS coordinate name to the `.proj` accessor:

In [5]:
ds_wgs84.proj("spatial_ref").crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

## CRS-aware alignment

The index of the CRS coordinate is used to compare or align xarray objects based on their CRS.

In [6]:
ds_pseudo_mercator = ds_wgs84.proj.assign_crs(spatial_ref="epsg:3857", allow_override=True)

Note the nice error message when trying to combine two datasets with different CRS (only works if the CRS coordinates have the same name), raised by Xarray and leveraging `pyproj.CRS`'s repr to output detailled information.

In [7]:
ds_wgs84 + ds_pseudo_mercator

MergeError: conflicting values/indexes on objects to be combined for coordinate 'spatial_ref'
first index: CRSIndex
<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

second index: CRSIndex
<Projected CRS: EPSG:3857>
Name: WGS 84 / Pseudo-Mercator
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: World between 85.06°S and 85.06°N.
- bounds: (-180.0, -85.06, 180.0, 85.06)
Coordinate Operation:
- name: Popular Visualisation Pseudo-Mercator
- method: Popular Visualisation Pseudo Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

first variable: <xarray.Variable ()> Size: 8B
array(0)
second variable: <xarray.Variable ()> Size: 8B
array(0

This also works seamlessly with `xarray.concat()`:

In [8]:
xr.concat([ds_wgs84.isel(space=[0, 1]), ds_wgs84.isel(space=[-1])], "space")

<xarray.Dataset> Size: 32B
Dimensions:      (space: 3)
Coordinates:
  * points       (space) object 24B POINT (171.61191601133118 -10.61889803036...
  * spatial_ref  int64 8B 0
Dimensions without coordinates: space
Data variables:
    *empty*
Indexes:
    points       GeometryIndex (crs=None)
    spatial_ref  CRSIndex (crs=EPSG:4326)

It is possible to combine heterogeneous geospatial Datasets (e.g., raster, vector, grid, mesh) as long as they all have a spatial reference coordinate with the same name and with a `CRSIndex`.

In [9]:
# lat-lon rectilinear grid

ds_grid = xr.Dataset(coords={"lat": np.linspace(-90, 90, 10), "lon": np.linspace(-180, 180, 20)})
ds_grid_wgs84 = ds_grid.proj.assign_crs(spatial_ref="epsg:4326")

# merge the lat-lon grid with the vector data cube
xr.merge([ds_wgs84, ds_grid_wgs84])

<xarray.Dataset> Size: 328B
Dimensions:      (space: 10, lat: 10, lon: 20)
Coordinates:
  * points       (space) object 80B POINT (171.61191601133118 -10.61889803036...
  * spatial_ref  int64 8B 0
  * lat          (lat) float64 80B -90.0 -70.0 -50.0 -30.0 ... 50.0 70.0 90.0
  * lon          (lon) float64 160B -180.0 -161.1 -142.1 ... 142.1 161.1 180.0
Dimensions without coordinates: space
Data variables:
    *empty*
Indexes:
    points       GeometryIndex (crs=None)
    spatial_ref  CRSIndex (crs=EPSG:4326)

## Unset the CRS

Just drop the CRS index and/or coordinate

In [10]:
ds_no_crs = ds_wgs84.drop_vars("spatial_ref")

ds_no_crs

<xarray.Dataset> Size: 80B
Dimensions:  (space: 10)
Coordinates:
  * points   (space) object 80B POINT (171.61191601133118 -10.618898030367944...
Dimensions without coordinates: space
Data variables:
    *empty*
Indexes:
    points   GeometryIndex (crs=None)

It is possible to combine datasets with and without a defined CRS. The resulting dataset will have the common CRS found among all datasets. 

In [11]:
ds_wgs84 + ds_no_crs

<xarray.Dataset> Size: 88B
Dimensions:      (space: 10)
Coordinates:
  * points       (space) object 80B POINT (171.61191601133118 -10.61889803036...
  * spatial_ref  int64 8B 0
Dimensions without coordinates: space
Data variables:
    *empty*
Indexes:
    points       GeometryIndex (crs=None)
    spatial_ref  CRSIndex (crs=EPSG:4326)